<a href="https://colab.research.google.com/github/Chiragdevadiga/Type--C/blob/main/Origami_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="mk0KG60ShShIw9dOeJZW")
project = rf.workspace("kyushu-university-dwxf1").project("fyp_20230907")
dataset = project.version(1).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to fyp_20230907-1 in yolov8:: 100%|██████████| 289/289 [00:00<00:00, 4474.81it/s]


In [87]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [88]:

# Define the dataset path
train_data_dir = '/content/fyp_20230907-1/train/'
valid_data_dir = '/content/fyp_20230907-1/valid/'

In [89]:
# Create ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [90]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [91]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)



Found 198 images belonging to 16 classes.


In [92]:
valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',

)

Found 84 images belonging to 16 classes.


In [93]:
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")

Number of classes: 16


In [99]:
# Build a simple CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [100]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [101]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=100  # you can adjust the number of epochs
)

Epoch 1/100
12/12 [==============================] - 4s 232ms/step - loss: 2.6241 - accuracy: 0.4615 - val_loss: 2.0163 - val_accuracy: 0.5000
Epoch 2/100
12/12 [==============================] - 3s 231ms/step - loss: 2.0952 - accuracy: 0.4945 - val_loss: 1.9021 - val_accuracy: 0.5250
Epoch 3/100
12/12 [==============================] - 3s 273ms/step - loss: 1.9844 - accuracy: 0.4945 - val_loss: 1.8671 - val_accuracy: 0.5000
Epoch 4/100
12/12 [==============================] - 5s 416ms/step - loss: 2.0008 - accuracy: 0.4890 - val_loss: 1.8605 - val_accuracy: 0.4750
Epoch 5/100
12/12 [==============================] - 4s 299ms/step - loss: 1.7861 - accuracy: 0.4948 - val_loss: 1.5558 - val_accuracy: 0.4750
Epoch 6/100
12/12 [==============================] - 3s 229ms/step - loss: 1.4805 - accuracy: 0.4835 - val_loss: 1.4693 - val_accuracy: 0.5000
Epoch 7/100
12/12 [==============================] - 4s 334ms/step - loss: 1.4562 - accuracy: 0.4890 - val_loss: 1.3090 - val_accuracy: 0.5250

In [103]:
# Save the model
model.save('origami_shape_classifier.h5')

In [104]:
import tensorflow as tf
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('origami_shape_classifier.h5')


In [105]:

# Path to your test image
test_image_path = '/content/hqdefault.jpg'


In [108]:

# Load and preprocess the test image
img = tf.keras.preprocessing.image.load_img(test_image_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the image


In [109]:

# Make predictions
predictions = model.predict(img_array)


1/1 [==============================] - 0s 20ms/step


In [110]:

# Map predicted class index to the actual class name
class_names = train_generator.class_indices
predicted_class = np.argmax(predictions)
predicted_class_name = [k for k, v in class_names.items() if v == predicted_class][0]


In [111]:

print(f"Predicted class index: {predicted_class}")
print(f"Predicted class name: {predicted_class_name}")


Predicted class index: 9
Predicted class name: rabbit
